In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [4]:
video_path = '/content/drive/My Drive/MemoryLane 416/Birthday of Zohaib.mp4'

In [5]:
cap=cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error opening video file")

# Line Detection

In [11]:
while True:
  ret,frame=cap.read()
  if not ret:
    print("video")
    break
  fliped=cv2.flip(frame,0)
  gray=cv2.cvtColor(fliped,cv2.COLOR_BGR2GRAY)
  edges=cv2.Canny(gray,100,200)
  lines=cv2.HoughLines(edges,1,np.pi/180,threshold=150)
  if lines is not None:
    for line in lines:
      rho,theta=line[0]
      a=np.cos(theta)
      b=np.sin(theta)
      x0=rho*a
      y0=rho*b
      x1=int(x0+1000*(-b))
      y1=int(y0+1000*(a))
      x2=int(x0-1000*(-b))
      y2=int(y0-1000*(a))
      cv2.line(fliped,(x1,y1),(x2,y2),(0,0,255),2)
  cv2_imshow(gray)

Output hidden; open in https://colab.research.google.com to view.


---

🔷 **Context:**
You have a line represented in polar coordinates (rho, theta). You need to **convert it into two points** to draw it in OpenCV.

---

```python
y0 = b * rho
```

✅ Calculates **y-coordinate of a base point** on the line using:

* `b` is sin(theta)
* Multiply it by `rho` to get **y0**.

---

```python
x1 = int(x0 + 1000*(-b))
```

✅ Calculates **x-coordinate of first end point** of the line:

* `x0` is base point.
* `-b` (negative sine) gives perpendicular direction.
* Multiply by a large number (1000) to make line long enough, then **add to x0**.

---

```python
y1 = int(y0 + 1000*(a))
```

✅ Calculates **y-coordinate of first end point** similarly:

* `a` is cos(theta).
* Multiply by 1000 for large length, add to y0.

---

```python
x2 = int(x0 - 1000*(-b))
```

✅ Calculates **x-coordinate of second end point**:

* Same logic as x1 but **subtracting** instead of adding to go in opposite direction.

---

```python
y2 = int(y0 - 1000*(a))
```

✅ Calculates **y-coordinate of second end point** similarly, subtracting.

---

🔷 **Why ±1000?**
Because each line from Hough Transform is infinite, and to **draw it visibly across the image**, you extend it both ways by a large value.

---

# **Circle Detection**

In [ ]:
while True:
  ret, frame = cap.read()
  if not ret:
    print("video")
    break
  fliped = cv2.flip(frame, 0)
  blur = cv2.GaussianBlur(fliped, (3,3), 0)
  circles = cv2.HoughCircles(cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY), cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=0, maxRadius=0)

  if circles is not None:
    circles = np.uint16(np.round(circles))
    for circle in circles[0, :]:
      x, y, r = circle
      cv2.circle(fliped, (x, y), r, (0, 255, 0), 2)

  cv2_imshow(fliped)



---

🔷 **5. Detect Circles using Hough Circle Transform**

```python
  circles = cv2.HoughCircles(blur, cv2.HOUGH_GRADIENT, 1, 20, param1=50, param2=30, minRadius=0, maxRadius=0)
```

✔️ Finds circles in the blurred image.

Here are the explanations in **easy words** for you:

---

1. blur

✅ Meaning:
The input image you give to cv2.HoughCircles().

✔️ Usually blurred using GaussianBlur to reduce noise, because noise can create false circle detections.

2. cv2.HOUGH_GRADIENT

✅ Meaning:
The detection method used by OpenCV for circle detection.

✔️ How it works:

It uses the gradient information (change in intensity) to detect circular shapes. This is efficient and accurate for detecting circles in images with clear edges.

3. dp (value = 1)

✅ Meaning:

Inverse ratio of the accumulator resolution to the image resolution.

✔️ Detailed Explanation:

If dp=1 ➔ the accumulator has the same resolution as the input image.

If dp=2 ➔ the accumulator is half the size (faster but may miss small details).

🔷 Accumulator: The internal image used by Hough transform to store voting results.

🔷 Resolution of accumulator:

It is like a hidden grid used to find circles – if you set dp=1, it’s the same size as the image, if dp=2, it’s half the size, making detection faster but possibly less accurate.

4. minDist (value = 20)

✅ Meaning:

Minimum distance between the centers of detected circles.

✔️ Detailed Explanation:

This prevents detecting multiple nearby circles that may actually be the same circle. For example, if set to 20 pixels, the centers of any two detected circles must be at least 20 pixels apart.

5. param1 (value = 50)

✅ Meaning:

First method-specific parameter. For cv2.HOUGH_GRADIENT, it is the higher threshold of the Canny edge detector.

✔️ Detailed Explanation:

Canny edge detection has two thresholds: low and high.

Edges stronger than param1 are kept as strong edges.

Edges between param2 and param1 are kept only if connected to strong edges.

In HoughCircles, param1 controls how strong edges should be to be used for circle detection.

6. param2 (value = 30)

✅ Meaning:

Second method-specific parameter. It is the threshold for center detection.

✔️ Detailed Explanation:

The smaller this value, the more false circles you will detect (algorithm becomes less strict).

The higher the value, the fewer circles detected, but detections are more reliable.

It controls how many votes (from the accumulator) are needed to accept a candidate circle.

7. minRadius, maxRadius (value = 0,0)

✅ Meaning:

Minimum and maximum radius of circles to detect.

✔️ Detailed Explanation:

If both are set to 0, the function tries to detect circles of any radius.

You can set specific values if you know the expected size of circles in your image, to speed up detection and improve accuracy.

---

🔷 **6. Loop Through Detected Circles**

```python
  for circle in circles:
    circle = np.uint16(np.round(circle))
```

✔️ Loops through each detected circle.
✔️ `np.round` rounds circle values, `np.uint16` converts them to integers.

✅ Note: `circles` returned is a 3D array, so you typically loop as:

```python
if circles is not None:
  circles = np.uint16(np.round(circles))
  for circle in circles[0, :]:
```

---

🔷 **7. Extract x, y, and radius**

```python
    x, y, r = circle[0]
```

✔️ Extracts **x-coordinate, y-coordinate, and radius** of the circle.

---

🔷 **8. Draw the Circle**

```python
    cv2.circle(fliped, (x, y), r, (0, 255, 0), 2)
```

✔️ Draws the circle on the flipped image.
✔️ Green color `(0,255,0)` and thickness `2`.

---

🔷 **9. Show Image (Incorrect Reference)**

```python
    cv2_imshow(gray)
```

⚠️ Shows `gray` image, which is undefined here. You probably intended:

```python
cv2_imshow(fliped)
```

✔️ This will display the flipped image with detected circles.

---